In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### import train and test data

In [2]:
x_train_origin = pd.read_csv('X_train.csv')
y_train_origin = pd.read_csv('y_train.csv')
x_test_origin = pd.read_csv('X_test_final.csv')
y_test_origin = pd.read_csv('y_test_random_final.csv')

#### preprocess

In [3]:
# merge x and y dataframe
df_train = pd.merge(x_train_origin, y_train_origin, on='ID')
df_test = pd.merge(x_test_origin, y_test_origin, on='ID')

In [4]:
# drop three columns
columns_to_drop = ['FR_NET_EXPORT', 'DE_NET_EXPORT', 'DE_FR_EXCHANGE']

df_train.drop(columns = columns_to_drop, inplace = True)
df_test.drop(columns = columns_to_drop, inplace = True)

In [5]:
df_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,...,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET
0,1054,206,FR,0.210099,-0.427458,0.606523,NaN,-0.692860,0.441238,-0.213766,...,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445,0.028313
1,2049,501,FR,-0.022399,-1.003452,0.022063,0.573520,1.130838,0.174773,0.426940,...,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365,-0.112516
2,1924,687,FR,1.395035,1.978665,-1.021305,0.622021,1.682587,2.351913,2.122241,...,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952,-0.180840
3,297,720,DE,-0.983324,-0.849198,0.839586,0.270870,-0.563230,0.487818,0.194659,...,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948,-0.260356
4,1101,818,FR,0.143807,-0.617038,0.924990,NaN,-0.990324,0.238693,-0.240862,...,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378,-0.071733


In [6]:
# fill nan using median value
medians = df_train.iloc[:,3:].median()

df_train.iloc[:,3:] = df_train.iloc[:,3:].fillna(medians)
df_test.iloc[:,3:] = df_test.iloc[:,3:].fillna(medians)

#### check input and output in one day_id

In [10]:
df_train[df_train['DAY_ID'] == 501][['ID', 'DAY_ID', 'COUNTRY', 'DE_CONSUMPTION', 'FR_CONSUMPTION', 'FR_DE_EXCHANGE', 'DE_NET_IMPORT', 'FR_NET_IMPORT',
                                    'DE_GAS', 'FR_GAS', 'DE_RAIN', 'FR_RAIN','DE_WIND', 'FR_WIND','TARGET']]

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,TARGET
1,2049,501,FR,-0.022399,-1.003452,0.022063,0.57352,1.130838,0.174773,0.42694,-1.2403,-0.770457,1.522331,0.828412,-0.112516
410,833,501,DE,-0.022399,-1.003452,0.022063,0.57352,1.130838,0.174773,0.42694,-1.2403,-0.770457,1.522331,0.828412,0.084221


#### split into x and y according to countries

In [7]:
df_train_fr = df_train[df_train['COUNTRY'] == 'FR']
df_train_de = df_train[df_train['COUNTRY'] == 'DE']
df_test_fr = df_test[df_test['COUNTRY'] == 'FR']
df_test_de = df_test[df_test['COUNTRY'] == 'DE']

In [8]:
df_train_fr.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,...,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET
0,1054,206,FR,0.210099,-0.427458,0.606523,0.306899,-0.692860,0.441238,-0.213766,...,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445,0.028313
1,2049,501,FR,-0.022399,-1.003452,0.022063,0.573520,1.130838,0.174773,0.426940,...,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365,-0.112516
2,1924,687,FR,1.395035,1.978665,-1.021305,0.622021,1.682587,2.351913,2.122241,...,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952,-0.180840
4,1101,818,FR,0.143807,-0.617038,0.924990,0.306899,-0.990324,0.238693,-0.240862,...,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378,-0.071733
5,1520,467,FR,-0.295296,-0.765120,0.717490,1.117139,0.200305,1.533595,0.306422,...,-0.962519,-0.193837,-0.871600,-0.917234,0.102046,0.472708,0.891049,0.861408,1.124457,0.932105


In [9]:
x_train_fr = df_train_fr.iloc[:,:-1]
y_train_fr = df_train_fr[['ID', 'TARGET']]
x_train_de = df_train_de.iloc[:,:-1]
y_train_de = df_train_de[['ID', 'TARGET']]

x_test_fr = df_test_fr.iloc[:,:-1]
y_test_fr = df_test_fr[['ID', 'TARGET']]
x_test_de = df_test_de.iloc[:,:-1]
y_test_de = df_test_de[['ID', 'TARGET']]

#### save as .csv

In [10]:
x_train_fr.to_csv('dataset/x_train_fr.csv', index = False)
y_train_fr.to_csv('dataset/y_train_fr.csv', index = False)
x_train_de.to_csv('dataset/x_train_de.csv', index = False)
y_train_de.to_csv('dataset/y_train_de.csv', index = False)

x_test_fr.to_csv('dataset/x_test_fr.csv', index = False)
y_test_fr.to_csv('dataset/y_test_fr.csv', index = False)
x_test_de.to_csv('dataset/x_test_de.csv', index = False)
y_test_de.to_csv('dataset/y_test_de.csv', index = False)